<a href="https://colab.research.google.com/github/Sxm-O/AI-Builders/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#fastai
!pip install -q fastbook==0.0.29

#ctgan
!pip install -q sdv==0.18.0

#review generation
!pip install -q transformers==4.27.3

#DuckDuckGo Search API
!pip install -q duckduckgo_search==5.3.0b4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 42.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 2.6 MB/s eta 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from fastbook import *
from fastai.vision.widgets import *
from tqdm.auto import tqdm
from duckduckgo_search import DDGS

#download from duckduckgo
def search_images_ddg(key, max_n=200):
     """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        (Adopted from https://github.com/deepanprabhu/duckduckgo-images-api)
     """
     url        = 'https://duckduckgo.com/'
     params     = {'q':key}
     res        = requests.post(url,data=params)
     searchObj  = re.search(r'vqd=([\d-]+)\&',res.text)
     if not searchObj: print('Token Parsing Failed !'); return
     requestUrl = url + 'i.js'
     headers    = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:71.0) Gecko/20100101 Firefox/71.0'}
     params     = (('l','us-en'),('o','json'),('q',key),('vqd',searchObj.group(1)),('f',',,,'),('p','1'),('v7exp','a'))
     urls       = []
     while True:
         try:
             res  = requests.get(requestUrl,headers=headers,params=params)
             data = json.loads(res.text)
             for obj in data['results']:
                 urls.append(obj['image'])
                 max_n = max_n - 1
                 if max_n < 1: return L(set(urls))     # dedupe
             if 'next' not in data: return L(set(urls))
             requestUrl = url + data['next']
         except:
             pass

def search_images_ddg_v2(key,max_n=200):
    """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        Note: Instead of using our own implementation, we using library instead!
    """
    responses = DDGS().images(
                          keywords=key,
                          region="wt-wt",
                          safesearch="off",
                          size=None,
                          color=None,
                          type_image=None,
                          layout=None,
                          license_image=None,
                          max_results=max_n,
                      )
    urls = [ r['image'] for r in responses ]
    return urls

In [4]:
#save ไว้ที่ไหน
path = Path('งูมีพิษ')
#เอารูปอะไรบ้าง (labels)
labels = ['Bungarus fasciatus', 'Ophiophagus hannah', 'Calloselasma rhodostoma', 'Cobra', "Siamese Russell's Viper",'Red necked keelback','Green pit viper']
#เอา label ละกี่รูป
MAX_N = 200

if not path.exists():
    path.mkdir()
for l in tqdm(labels):
    dest = (path/l)
    dest.mkdir(exist_ok=True)
    results = search_images_ddg_v2(l, max_n=MAX_N)
    download_images(dest, urls=results)

  0%|          | 0/7 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [5]:
#เช็คว่าได้รูปอะไรบ้าง
fns = get_image_files(path)
fns

(#956) [Path('งูมีพิษ/Calloselasma rhodostoma/2c31dc06-e25f-49a5-9245-dbf827645fe8.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/3f78f36f-adfa-4811-bc40-5d9ae0d831f2.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/9c33a86f-7291-459d-a6cb-935e3d22cc9e.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/fb1cf980-7f1f-4547-8396-352edd8ff0b8.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/133dbe95-ffbc-46a8-81b7-81a251a729ec.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/3eec5a89-1067-4a16-b61a-c1eb57132a30.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/36c2935a-efc1-45a0-90c7-411ed6376ea3.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/eba8a473-aa47-4c3b-be2c-da57614296be.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/99071e3e-6d45-49b2-9339-fc95ec8623c9.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/3b9a3a00-6b15-4713-9b30-fcf55b20f6f8.jpg')...]

In [6]:
#ดูว่าหารูปไหนไม่เจอบ้าง
failed = verify_images(fns)
failed

(#37) [Path('งูมีพิษ/Calloselasma rhodostoma/31717e95-aa10-44b9-80a0-1837e6d1941b.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/0513d3b7-3b93-4d6d-baab-48af60080ed0.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/c58acd2b-7caa-4531-b8ad-c57e0536b11e.jpg'),Path('งูมีพิษ/Calloselasma rhodostoma/dfcaeaf7-8c9f-4dc9-ab0a-05506f1a9e85.jpg'),Path('งูมีพิษ/Ophiophagus hannah/c478043c-0acd-485e-8c9e-6268958bb952.jpg'),Path('งูมีพิษ/Ophiophagus hannah/69c9ac9c-5f13-4b70-b98e-98e41ca6b389.jpg'),Path('งูมีพิษ/Ophiophagus hannah/5ad98c27-2a3b-4d9e-b5a2-ca754e4fc2d7.jpg'),Path('งูมีพิษ/Green pit viper/6ac3b6d4-e63d-48b5-ae6f-0ac65b748923.jpg'),Path('งูมีพิษ/Green pit viper/2c72111f-f163-484b-b1ab-3c2da24f9763.jpg'),Path('งูมีพิษ/Green pit viper/07b014e1-70d8-4d89-bb7a-a88bf755046c.jpg')...]

In [7]:
#นำรูปที่หาไม่เจอออก
failed.map(Path.unlink)

(#37) [None,None,None,None,None,None,None,None,None,None...]

In [8]:
#save ไว้ที่ไหน
path = Path('งูไม่มีพิษ')
#เอารูปอะไรบ้าง (labels)
labels = ['Cylindrophis ruffus', 'Coelognathus radiatus', 'Oligodon barroni', 'Oriental whip snake', 'Reticulated python', 'Sunbeam snake']
#เอา label ละกี่รูป
MAX_N = 200

if not path.exists():
    path.mkdir()
for l in tqdm(labels):
    dest = (path/l)
    dest.mkdir(exist_ok=True)
    results = search_images_ddg_v2(l, max_n=MAX_N)
    download_images(dest, urls=results)

  0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
#เช็คว่าได้รูปอะไรบ้าง
fns = get_image_files(path)
fns

(#956) [Path('งูไม่มีพิษ/Oriental whip snake/12e02c60-c156-4097-8fb7-69100693a00d.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/a9e93f7c-0fba-4e96-a405-5d59a2bd580c.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/802b073e-3dd9-4e76-8e44-0f7f20a04a1d.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/6bd2848a-2cfc-44d3-b41c-4714c28a0890.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/18a4c90d-261c-4a05-ae97-7acc1d164ef2.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/ad61c1aa-a4ad-46f0-ac13-eec5c2b68353.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/26d39777-aa98-44af-a010-3e0afe10acb6.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/d0d46907-bc49-4cdd-81ce-4c7d285e7aa3.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/855a1070-3cc2-4358-a9b7-9edb548ec879.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/02737595-13ac-4f96-842f-4cd0eaf86638.jpg')...]

In [10]:
#ดูว่าหารูปไหนไม่เจอบ้าง
failed = verify_images(fns)
failed

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


(#14) [Path('งูไม่มีพิษ/Oriental whip snake/b1768736-39f8-4982-b0b2-84d2715851e4.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/f5830418-e655-4470-982c-03400782ad6e.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/959227d9-4b75-47bd-9554-2b1695cabbc9.jpg'),Path('งูไม่มีพิษ/Oriental whip snake/42eb3565-2d1e-4551-96b7-57c7c494d2df.jpg'),Path('งูไม่มีพิษ/Reticulated python/1e2a38df-dff6-45c4-8832-3cc26fa779f5.jpg'),Path('งูไม่มีพิษ/Reticulated python/8fde79e1-ff32-41ed-83a7-6543016ea302.jpg'),Path('งูไม่มีพิษ/Reticulated python/66f86f36-d54c-478a-be69-7bee8bd8d8d5.jpg'),Path('งูไม่มีพิษ/Cylindrophis ruffus/df5b2cd2-e766-4625-a7f2-1873d4fcda86.jpg'),Path('งูไม่มีพิษ/Sunbeam snake/8d2d72d1-c511-4565-a202-4dbd64d8cf72.jpg'),Path('งูไม่มีพิษ/Sunbeam snake/02c7d960-9a4a-46c5-998b-9c68bfa9a3ce.png')...]

In [11]:
#นำรูปที่หาไม่เจอออก
failed.map(Path.unlink)

(#14) [None,None,None,None,None,None,None,None,None,None...]